In [1]:
import json
import time

import requests
import pymongo

In [2]:
client = pymongo.MongoClient('localhost', 27017)
db = client['league']
players_db = db['players']
matches_db = db['matches']

In [3]:
TOKEN = 'TOKEN'
PLATFORMS = ['BR1', 'EUN1', 'EUW1', 'JP1', 'KR', 'LA1', 'LA2', 'NA1', 'OC1', 'TR1', 'RU']
TIERS = ['CHALLENGER', 'GRANDMASTER', 'MASTER', 'DIAMOND', 'PLATINUM', 'GOLD', 'SILVER', 'BRONZE', 'IRON']
ACCOUNT_IDS = list() # To get match history
GAME_IDS = list() # To get match info
MATCH_COLLECTION = list() # To collect all matches

In [4]:
champions_json = json.loads(open('data/en_US/champion.json', 'r', encoding='utf-8').read())

championsId = dict()

for champ in champions_json['data']:
    id = champions_json['data'][champ]['key']
    name = champions_json['data'][champ]['name']
    
    championsId.update(
        {
            id: name
        }
    )

In [5]:
spells_json = json.loads(open('data/en_US/summoner.json', 'r', encoding='utf-8').read())

spellsId = dict()

for spell in spells_json['data']:
    id = spells_json['data'][spell]['key']
    name = spells_json['data'][spell]['name']
    
    spellsId.update(
        {
            id: name
        }
    )

In [6]:
items_json = json.loads(open('data/en_US/item.json', 'r', encoding='utf-8').read())

itemsId = dict(
    {
        '0': {}
    }
)

for item in items_json['data']:
    id = item
    name = items_json['data'][item]['name']
    stats = items_json['data'][item]['stats']
    tags = items_json['data'][item]['tags']
    gold = items_json['data'][item]['gold']
    
    try:
        _from = items_json['data'][item]['from']
    except Exception:
        _from = []
    else:
        for index, i in enumerate(_from):
            _from[index] = items_json['data'][i]['name']
       
    try:
        into = items_json['data'][item]['into']
    except Exception:
        into = []
    else:
        for index, i in enumerate(into):
            into[index] = items_json['data'][i]['name']
    
    itemsId.update(
        {
            id: {
                'name': name,
                'from': _from,
                'into': into,
                'stats': stats,
                'tags': tags,
                'gold': gold
            }
        }
    )

In [7]:
runes_json = json.loads(open('data/en_US/runesReforged.json', 'r', encoding='utf-8').read())
perks_json = json.loads(open('data/en_US/perks.json', 'r', encoding='utf-8').read())

perksId = dict()

for rune in runes_json:
    id = str(rune['id'])
    name = rune['name']

    perksId.update(
        {
            id: name
        }
    )
    
for perk in perks_json:
    id = str(perk['id'])
    name = perk['name']
    
    perksId.update(
        {
            id: name
        }
    )

In [8]:
def parse_match(match: dict) -> dict:
    # Teams
    match['teams'][0]['teamId'], match['teams'][1]['teamId'] = ('Blue', 'Red')
    
    # Banned champions
    for team in (0, 1): # Blue, Red
        for banned_champ in match['teams'][team]['bans']:
            if banned_champ['championId'] == -1:
                banned_champ['championId'] = 'None'
            else:
                banned_champ['championId'] = championsId[str(banned_champ['championId'])]
            
    # Players
    for player in match['participants']:
        player['teamId'] = 'Blue' if player['teamId'] == 100 else 'Red'
        player['championId'] = championsId[str(player['championId'])]
        
        player['spell1Id'] = spellsId[str(player['spell1Id'])]
        player['spell2Id'] = spellsId[str(player['spell2Id'])]
        
        for i in range(7):
            player['stats'][f'item{i}'] = itemsId[str(player['stats'][f'item{i}'])]
        
        for i in range(6):
            player['stats'][f'perk{i}'] = perksId[str(player['stats'][f'perk{i}'])]

        player['stats']['perkPrimaryStyle'] = perksId[str(player['stats']['perkPrimaryStyle'])]
        player['stats']['perkSubStyle'] = perksId[str(player['stats']['perkSubStyle'])]

        player['stats']['statPerk0'] = perksId[str(player['stats']['statPerk0'])]
        player['stats']['statPerk1'] = perksId[str(player['stats']['statPerk1'])]
        player['stats']['statPerk2'] = perksId[str(player['stats']['statPerk2'])]
        
    return match

In [9]:
s = requests.Session()
s.proxies = {'http': 'http://61.233.25.166:80'}

In [10]:
def get_account_ids(platform: str, tier: str) -> None:
    players = players_db.find({'platform': platform, 'tier': tier}, no_cursor_timeout=True)

    for player in players:
        ACCOUNT_IDS.append(player['accountId'])

    players.close()

In [11]:
def get_match_history(platform: str) -> None:
    counter = 0
    accountIds = ACCOUNT_IDS.copy()
    
    for accountId in accountIds:
        counter += 1
        print(f'{counter}/{len(accountIds)}', end='\r')
        
        url = f'https://{platform}.api.riotgames.com/lol/match/v4/matchlists/by-account/{accountId}?queue=420&endIndex=10'
        
        time.sleep(1.2)
        
        try:
            r = s.get(url, headers={'X-Riot-Token': TOKEN})
        except Exception as e:
            print('CONNECTION ERROR')
            continue
        
        if r.status_code == 200:    
            for match in r.json()['matches']:
                GAME_IDS.append(match['gameId'])
            
            ACCOUNT_IDS.remove(accountId)
            
        elif r.status_code == 429:
            print('\nRATE LIMIT EXCEEDED')
            time.sleep(8)
        
        elif r.status_code == 404:
            print('\nMATCH HISTORY NOT FOUND')
            ACCOUNT_IDS.remove(accountId)
        
        elif r.status_code == 403:
            print('\nINVALID TOKEN')
            return
    
    print() # Aesthetic sake

In [12]:
def get_matches(platform: str) -> None:
    counter = 0
    gameIds = GAME_IDS.copy()
    
    for gameId in gameIds:
        counter += 1
        print(f'\r{counter}/{len(gameIds)}', end='\r')
        
        url = f'https://{platform}.api.riotgames.com/lol/match/v4/matches/{gameId}'
        
        time.sleep(1.2)
        
        try:
            r = s.get(url, headers={'X-Riot-Token': TOKEN})
        except Exception as e:
            print('CONNECTION ERROR')
            continue
            
        if r.status_code == 200:
            try: # For when the API is bad and the response is missing keywords
                match = parse_match(r.json())
            except KeyError:
                print(f'MATCH {gameId} IGNORED')
                continue
            else:
                MATCH_COLLECTION.append(match)
            finally:
                GAME_IDS.remove(gameId)
        
        elif r.status_code == 429:
            print('\nRATE LIMIT EXCEEDED')
            time.sleep(8)
        
        elif r.status_code == 404:
            print('\nMATCH NOT FOUND')
            GAME_IDS.remove(gameId)
        
        elif r.status_code == 403:
            print('\nINVALID TOKEN')
            return
    
    print() # Aesthetic sake

In [13]:
for platform in PLATFORMS:
    for tier in TIERS:
        print(f'Collecting matches from {platform} - {tier}')
        
        ACCOUNT_IDS = list()        
        GAME_IDS = list()
        MATCH_COLLECTION = list()
        
        get_account_ids(platform, tier)
        total = 10 * len(ACCOUNT_IDS) # 10 matches from every player
        
        print(f'Collecting match IDs..')
        
        while ACCOUNT_IDS:
            get_match_history(platform)
        
        print(f'Collecting matches..')

        while GAME_IDS:
            get_matches(platform)
        
        matches_db.insert_many(MATCH_COLLECTION)
        print(f'{len(MATCH_COLLECTION)}/{total} matches saved.')